In [ ]:
from checkpoint import openpmdcopy, print_attributes

import math
import matplotlib.pyplot as plt
from numba import cuda, jit
import numpy as np
import openpmd_api as io
import scipy.constants
from scipy.ndimage import gaussian_filter
import time


## Things to do in the future

* specify a cutoff, behind all fields are written as 0 instead of a small value e.g. 1e-10 * field_max
* multi GPU / multi Node calculations  field size are limited by GPU memory size

## Things to note
* The Convolutional PML fields aren't changed, which might lead to unexpected simulations
* Only use this initialization on the 0 timestep (for now)

## Set path to checkpoint

In [ ]:

# you can use one checkpoint explicitly checkpoint_0.h5
# or a time series with checkpoint_%T.h5
# when giving an explicit checkpoint you can ignore the warning

read_file = "/bigdata/hplsim/scratch/spreng88/runs/bunch/nonR_setup2/simOutput/checkpoints/checkpoint_%T.bp"
output_file = "/bigdata/hplsim/scratch/spreng88/runs/write_%T.bp"

files = openpmdcopy(read_file, output_file)

## General setup

In [ ]:

# add field to iteration 0
input_iteration = files.read_series.iterations[0] 
output_iteration = files.write_series.iterations[0]

# Read attributes of the simulation that might be needed later
# Everything is calculated in PIConGPU units
# For SI units: length_si = value_pic * unit_length
cell_depth = input_iteration.get_attribute("cell_depth")          # z
cell_height = input_iteration.get_attribute("cell_height")        # y
cell_width = input_iteration.get_attribute("cell_width")          # x

unit_efield = input_iteration.get_attribute("unit_efield")
unit_bfield = input_iteration.get_attribute("unit_bfield")
unit_charge = input_iteration.get_attribute("unit_charge")
unit_mass = input_iteration.get_attribute("unit_mass")
unit_speed = input_iteration.get_attribute("unit_speed")
unit_length = input_iteration.get_attribute("unit_length")
unit_time = input_iteration.get_attribute("unit_time")

pi = scipy.constants.pi
c = scipy.constants.c / unit_speed
eps0 = input_iteration.get_attribute("eps0")
mue0 = input_iteration.get_attribute("mue0")

## Cuda functions

In [ ]:

@cuda.jit(device=True)
def particleV(px, py, pz, mass):
    """
    calculate particle speed from momentum
    """
    
    m2p2 = math.sqrt( (mass)**2 + px**2 + py**2 + pz**2)
    
    vx = px / m2p2 * c
    vy = py / m2p2 * c
    vz = pz / m2p2 * c
    
    return vx, vy, vz


@cuda.jit(device=True)
def param1(rx,ry,rz, rqx,rqy,rqz, vx,vy,vz):
    """
    calculate some parameters that could be reused
    r: position of field calculation
    rq: position of charge q at time t
    rq_tr: position of charge at retarded time tr
    """
    
    #
    # solution equation:
    # dt = t - tr   = time - retarded_time
    # r where we want the field
    # rq position of charge
    # |c*dt| = |r - (rq - v*dt)|
    # solve for dt:
    # => (c^2 - |v|^2) * dt^2 - 2(r*v - rq*v) * dt + 2 * r*rq - |r|^2 - |rq|^2 = 0
    #
    a = (c**2 - (vx**2 + vy**2 + vz**2))
    b = -2 * ((rx-rqx)*vx + (ry-rqy)*vy + (rz-rqz)*vz)
    d = 2 * (rx*rqx + ry*rqy + rz*rqz) - (rx**2 + ry**2 + rz**2) - (rqx**2 + rqy**2 + rqz**2)
    dt = (-b + math.sqrt(b**2 - 4*a*d)) / (2*a)
    
    rq_trx = rqx - vx * dt
    rq_try = rqy - vy * dt
    rq_trz = rqz - vz * dt

    dvx = rx - rq_trx
    dvy = ry - rq_try
    dvz = rz - rq_trz
    
    distance = math.sqrt(dvx**2 + dvy**2 + dvz**2)
    
    if distance == 0:
        distance = -1
    
    #n =  distanceVec / distance
    nx = dvx / distance
    ny = dvy / distance
    nz = dvz / distance
    
    return nx, ny, nz, distance, dvx, dvy, dvz #n, distance, distanceVec


@cuda.jit(device=True)
def retardedEFieldParallel(q, nx, ny, nz, distance, dvx, dvy, dvz, vx, vy, vz):
    """
    field for one position
    q: charge
    nx, ny, nz: unit vector to retarded position
    distance: distance to retarded position
    dvx, dvy, dvz: distance Vector to retarded position
    vx, vy, vz: velocity vector of the charge
    """
    
    if distance == -1:
        nx = 1
        ny = 0
        xz = 0
    
    factor = q / (4 * pi * eps0)
    
    ux = c * nx - vx
    uy = c * ny - vy
    uz = c * nz - vz
    

    scalar = factor * distance / (dvx*ux + dvy*uy + dvz*uz)**3 * (c**2 - (vx**2 + vy**2 + vz**2))
    
    return  scalar * ux, scalar * uy, scalar * uz


@cuda.jit(device=True)
def _returnFields(q, rx,rex,ry,rey,rz,rez, rqx,rqy,rqz, px,py,pz, mass):
    """calculate the fields for the position,
    currently ignoring the offset between the grid corner and field points
    because we will use smoothing in the end over the field anyway"""
    
    vx, vy, vz = particleV(px, py, pz, mass)
    
    ## E Field ------------------------------------------------------------
    nx, ny, nz, distancez, dvx, dvy, dvz = param1(rx, ry, rz, rqx, rqy, rqz, vx,vy,vz)
    ex, ey, ez = retardedEFieldParallel(q, nx, ny, nz, distancez, dvx, dvy, dvz , vx, vy, vz)
    

    
    ## B Field ------------------------------------------------------------
    bx = ( ny * ez - nz * ey ) / c
    by = ( nz * ex - nx * ez ) / c
    bz = ( nx * ey - ny * ex ) / c
    
    return ex, ey, ez, bx, by, bz


@cuda.jit(device=True)
def returnFields(q, rx,rex,ry,rey,rz,rez, rqx,rqy,rqz, px,py,pz, mass):
    """calculate the fields for the position,
    currently ignoring the offset between the grid corner and field points
    because we will use smoothing in the end over the field anyway"""

    vx, vy, vz = particleV(px, py, pz, mass)

    ## E Field ------------------------------------------------------------
    nx1, ny1, nz1, distancez, dvx, dvy, dvz = param1(rex, ry, rz, rqx, rqy, rqz, vx,vy,vz)
    ex1, ey1, ez1 = retardedEFieldParallel(q, nx1, ny1, nz1, distancez, dvx, dvy, dvz , vx, vy, vz)

    nx2, ny2, nz2, distancez, dvx, dvy, dvz = param1(rx, rey, rz, rqx, rqy, rqz, vx,vy,vz)
    ex2, ey2, ez2 = retardedEFieldParallel(q, nx2, ny2, nz2, distancez, dvx, dvy, dvz , vx, vy, vz)

    nx3, ny3, nz3, distancez, dvx, dvy, dvz = param1(rx, ry, rez, rqx, rqy, rqz, vx,vy,vz)
    ex3, ey3, ez3 = retardedEFieldParallel(q, nx3, ny3, nz3, distancez, dvx, dvy, dvz , vx, vy, vz)

    nx4, ny4, nz4, distancez, dvx, dvy, dvz = param1(rex, rey, rz, rqx, rqy, rqz, vx,vy,vz)
    ex4, ey4, ez4 = retardedEFieldParallel(q, nx4, ny4, nz4, distancez, dvx, dvy, dvz , vx, vy, vz)

    nx5, ny5, nz5, distancez, dvx, dvy, dvz = param1(rx, rey, rez, rqx, rqy, rqz, vx,vy,vz)
    ex5, ey5, ez5 = retardedEFieldParallel(q, nx5, ny5, nz5, distancez, dvx, dvy, dvz , vx, vy, vz)

    nx6, ny6, nz6, distancez, dvx, dvy, dvz = param1(rex, ry, rez, rqx, rqy, rqz, vx,vy,vz)
    ex6, ey6, ez6 = retardedEFieldParallel(q, nx6, ny6, nz6, distancez, dvx, dvy, dvz , vx, vy, vz)

    ex = ex1
    ey = ey2
    ez = ez3


    ## B Field ------------------------------------------------------------
    bx = ( ny5 * ez5 - nz5 * ey5 ) / c
    by = ( nz6 * ex6 - nx6 * ez6 ) / c
    bz = ( nx4 * ey4 - ny4 * ex4 ) / c

    return ex, ey, ez, bx, by, bz


In [ ]:
@cuda.jit
def particleParallel(Ex,Ey,Ez, Bx,By,Bz, q_, rqx_,rqy_,rqz_, px_,py_,pz_, mass_, weighting_, xdim, ydim, zdim, particleCount, chunk_offset):
    """
    calculate all particles in parallel, loop over every point in the field
    
    Ex,Ey,Ez E field where new field is added
    Bx,By,Bz B -"-
    
    _ is on all input arrays read from checkpoint 
    q_ array from checkpoint with charges
    rq_ position data of particles
    p_ momentum of particles
    weighting_ macro particle weighting
    
    xdim, ydim, zdim length of dimension of Ex,Bx, ...
    Ex/Bx need to be 1D for atomic add
    """
    
    tix = cuda.threadIdx.x
    bix = cuda.blockIdx.x
    bdx = cuda.blockDim.x
    
    index = tix + bix * bdx    # particle index
    index = index
    
    if index < particleCount:
    
        q = q_[index] * weighting_[index]
        rqx = rqx_[index] * cell_width
        rqy = rqy_[index] * cell_height
        rqz = rqz_[index] * cell_depth
        px = px_[index]
        py = py_[index]
        pz = pz_[index]
        mass = mass_[index] * weighting_[index]

        for x in range(xdim):

            rx = (x + chunk_offset[2]) * cell_width
            rex = (x + chunk_offset[2] + 0.5) * cell_width

            for y in range(ydim):

                ry = (y + chunk_offset[1]) * cell_height
                rey = (y + chunk_offset[1] + 0.5) * cell_height

                for z in range(zdim):    
                    
                    rz = (z + chunk_offset[0]) * cell_depth
                    rez = (z + chunk_offset[0] + 0.5) * cell_depth

                    fieldIndex = x + y * xdim + z * xdim * ydim
                    
                    ex, ey, ez, bx, by, bz = returnFields(q, rx,rex,ry,rey,rz,rez, rqx,rqy,rqz, px,py,pz, mass)
                    
                    cuda.atomic.add(Ex, fieldIndex, ex)
                    cuda.atomic.add(Ey, fieldIndex, ey)
                    cuda.atomic.add(Ez, fieldIndex, ez)

                    cuda.atomic.add(Bx, fieldIndex, bx)
                    cuda.atomic.add(By, fieldIndex, by)
                    cuda.atomic.add(Bz, fieldIndex, bz)

In [ ]:
@cuda.jit
def FieldParallel(Ex,Ey,Ez, Bx,By,Bz, q_, rqx_,rqy_,rqz_, px_,py_,pz_, mass_, weighting_, xdim, ydim, zdim, particleCount, chunk_offset):
    """
    Calculate all field points in parallel, loop over all particles
    
    Ex,Ey,Ez E field where new field is added
    Bx,By,Bz B -"-
    
    a trailing _ is on all input arrays read from checkpoint 
    q_ array from checkpoint with charges
    rq_ position data of particles
    p_ momentum of particles
    
    xdim, ydim, zdim length of dimension of Ex,Bx, ...
    Ex/Bx need to be 1D for atomic add
    """
    tix = cuda.threadIdx.x
    bix = cuda.blockIdx.x
    bdx = cuda.blockDim.x
    
    fieldIndex = tix + bix * bdx    # field index
    
    # calculate 3d coordinate from 1d field coordinate array
    # fieldIndex = x + y * xdim + z * xdim * ydim
    z = math.floor( fieldIndex / (xdim*ydim) )
    y = math.floor( (fieldIndex - z * xdim*ydim) / xdim )
    x = fieldIndex - z * xdim*ydim - y * xdim
    
    x = x + chunk_offset[2]
    y = y + chunk_offset[1]
    z = z + chunk_offset[0]
    
    if fieldIndex < xdim*ydim*zdim:
        
        #for index in range(particleCount):
        # or add filter if you only want the field of some particles of that species
        for index in range(particleCount):
        
            q = q_[index] * weighting_[index]
            rqx = rqx_[index] * cell_width
            rqy = rqy_[index] * cell_height
            rqz = rqz_[index] * cell_depth
            px = px_[index]
            py = py_[index]
            pz = pz_[index]
            mass = mass_[index] * weighting_[index]
            
            rx = x * cell_width
            rex = (x + 0.5) * cell_width
            
            ry = y * cell_height
            rey = (y + 0.5) * cell_height
            
            rz = z * cell_depth
            rez = (z + 0.5) * cell_depth
            
            ex, ey, ez, bx, by, bz = returnFields(q, rx,rex,ry,rey,rz,rez, rqx,rqy,rqz, px,py,pz, mass)

            cuda.atomic.add(Ex, fieldIndex, ex)
            cuda.atomic.add(Ey, fieldIndex, ey)
            cuda.atomic.add(Ez, fieldIndex, ez)

            cuda.atomic.add(Bx, fieldIndex, bx)
            cuda.atomic.add(By, fieldIndex, by)
            cuda.atomic.add(Bz, fieldIndex, bz)
            

In [ ]:
files.copy_series_data()

In [ ]:
def smoothFields(ex, ey, ez, bx, by, bz, sigma):
    ex = gaussian_filter(ex.reshape(zdim, ydim, xdim), sigma)
    ey = gaussian_filter(ey.reshape(zdim, ydim, xdim), sigma)
    ez = gaussian_filter(ez.reshape(zdim, ydim, xdim), sigma)
    bx = gaussian_filter(bx.reshape(zdim, ydim, xdim), sigma)
    by = gaussian_filter(by.reshape(zdim, ydim, xdim), sigma)
    bz = gaussian_filter(bz.reshape(zdim, ydim, xdim), sigma)

In [ ]:
def CIC_assignment(x):
    return (1-np.abs(x)) * ( np.abs(x) < 1 )

def TSC_assignment(x):
    return  (3/4 - x**2) * (np.abs(x) < 1/2) + 1/2*(3/2 - np.abs(x))**2 * ((np.abs(x)<3/2) & (np.abs(x)>1/2) )

def PQS_assignment(x):
    return 1/6* (4 - 6 * x**2 + 3 * np.abs(x)**3) * (np.abs(x) < 1) + 1/6 * (2-np.abs(x))**3 * ((np.abs(x)<2) & (np.abs(x)>1))

## load particle data from the checkpoint
define the species identifier string, as in the simulation

In [ ]:
species = "b"

xpos_incell = input_iteration.particles[species]["position"]["x"][:]
ypos_incell = input_iteration.particles[species]["position"]["y"][:]
zpos_incell = input_iteration.particles[species]["position"]["z"][:]
xpos_offset = input_iteration.particles[species]["positionOffset"]["x"][:]
ypos_offset = input_iteration.particles[species]["positionOffset"]["y"][:]
zpos_offset = input_iteration.particles[species]["positionOffset"]["z"][:]
momentumx = input_iteration.particles[species]["momentum"]["x"][:]
momentumy = input_iteration.particles[species]["momentum"]["y"][:]
momentumz = input_iteration.particles[species]["momentum"]["z"][:]
weightings = input_iteration.particles[species]["weighting"][io.Record_Component.SCALAR][:]
charge = input_iteration.particles[species]["charge"][io.Record_Component.SCALAR][:]
mass = input_iteration.particles[species]["mass"][io.Record_Component.SCALAR][:]

files.read_series.flush()

xpos = xpos_incell + np.float32(xpos_offset)
ypos = ypos_incell + np.float32(ypos_offset)
zpos = zpos_incell + np.float32(zpos_offset)

# free some memory
del xpos_incell, ypos_incell, zpos_incell
xpos_offset, ypos_offset, zpos_offset

particleCount = len(mass)

## output to check particleCount and gridsize

In [ ]:
shape_assignment_function = TSC_assignment
n = 5
#particleCount = 1
_xpos = []
_ypos = []
_zpos = []
_charge = []
_momentumx = []
_momentumy = []
_momentumz = []
_weighting = []
_mass = []

def apply_shape():
    for pos_id in range(particleCount):
        x = xpos[pos_id]
        y = ypos[pos_id]
        z = zpos[pos_id]
        #print(x, y ,z)
        dim = np.arange(n)
        charge_density = 0 # np.zeros(shape=(n, n, n))
        
        x_off = xpos_offset[pos_id]
        y_off = ypos_offset[pos_id]
        z_off = zpos_offset[pos_id]
        counter = 0
        for zz in dim:
            for yy in dim:
                for xx in dim:
                    index = xx + yy*n + zz*n*n
                    charge_density = (shape_assignment_function(xx - x + x_off - 2) *
                                      shape_assignment_function(yy - y + y_off - 2) *
                                      shape_assignment_function(zz - z + z_off - 2) *
                                      charge[pos_id])
                  #  print(shape_assignment_function(xx - x + x_off - 2),
                   #       shape_assignment_function(yy - y + y_off - 2),
                    #      shape_assignment_function(zz - z + y_off - 2))
                    if charge_density != 0:
                        #print(charge_density, charge_density != 0)
                        _xpos.append(x_off + xx - 2)
                        _ypos.append(y_off + yy - 2)
                        _zpos.append(z_off + zz - 2)
                        _momentumx.append(momentumx[pos_id])
                        _momentumy.append(momentumy[pos_id])
                        _momentumz.append(momentumz[pos_id])
                        _weighting.append(weightings[pos_id])
                        _mass.append(mass[pos_id])
                        _charge.append(charge_density)


apply_shape()

#print(_xpos)
#print(_ypos)
#print(_zpos)
#print(_charge)
#print(_momentumx)
#print(_momentumy)
#print(_momentumz)
#print(_weighting)
#print(_mass)

xpos = _xpos
ypos = _ypos
zpos = _zpos
charge = _charge
momentumx = _momentumx
momentumy = _momentumy
momentumz = _momentumz
weighting = _weighting
mass = _mass


In [ ]:
print("particleCount:", particleCount, len(mass))
input_field_shape = files.input_iteration.meshes["E"]["x"].shape
xdim_total = input_field_shape[2]
ydim_total = input_field_shape[1]
zdim_total = input_field_shape[0]
input_field_shape_total = xdim_total * ydim_total * zdim_total
print("gridSize (z,y,x, total):", input_field_shape, input_field_shape_total)

field_dtype = files.input_iteration.meshes['E']['x'].dtype
field_dataset = io.Dataset( field_dtype, input_field_shape)

for field_name in ['E', 'B']:
    files.copy_attributes(files.input_iteration.meshes[field_name],
                          files.output_iteration.meshes[field_name])
    for component in ['x', 'y', 'z']:
        files.copy_attributes(files.input_iteration.meshes[field_name][component],
                              files.output_iteration.meshes[field_name][component])
        files.output_iteration.meshes[field_name][component].reset_dataset(field_dataset)

## calculate the fields

set the chunk_size to the number of cells if the gpu has enough memory

In [ ]:

chunk_size = np.array([768, 512, 768], dtype=np.int32)  # z y x
xdim = chunk_size[2]
ydim = chunk_size[1]
zdim = chunk_size[0]
shape = xdim * ydim * zdim

min_memory_device = shape * 24 + particleCount * 56
print("device needs a minimum memory of:",min_memory_device, "Bytes = {:.5} GB".format(min_memory_device*1e-9))

chunk_offset = np.array([0, 128, 0], dtype=np.int32)          # offset for chunk (z, y, z)


In [ ]:

#if particleCount <= shape:    # maybe add a constant, but needs testing for optimal choise
#    # calculate every field position in parallel (loop over the particles)
#    # few particles, many field positions
#    blockdim = 256                                              # number of threads per block (multiple 32 for optimal speed)
#    griddim = math.ceil(shape / blockdim)                       # number of blocks in the grid
#    cudaFunc = FieldParallel
#    print("Fields in parallel")
#    
#else:
#    # calculate every particle in parallel (loop over the entire field)
#    # many particles, few field positions
#    blockdim = 32  #256                                         # number of threads per block (multiple 32 for optimal speed)
#    griddim = math.ceil(particleCount / blockdim)         # number of blocks in the grid
#    cudaFunc = particleParallel
#    print("particles in parallel")
particleCount = len(charge)
    
blockdim = 256                                   # number of threads per block (multiple 32 for optimal speed)
griddim = math.ceil(shape / blockdim)            # number of blocks in the grid
cudaFunc = FieldParallel
    
print("particles to be processed:", particleCount)

# for test on a smaller number of particles
#blockdim = 1
#griddim = 1

array_dtype = np.float32                            # dtype for arrays
ex = np.zeros(shape, dtype=array_dtype)
ey = np.zeros(shape, dtype=array_dtype)
ez = np.zeros(shape, dtype=array_dtype)
bx = np.zeros(shape, dtype=array_dtype)
by = np.zeros(shape, dtype=array_dtype)
bz = np.zeros(shape, dtype=array_dtype)


# All variables with _d at the and are for device variables
print("allocate/copy field/data arrays to device")
ex_d = cuda.to_device(ex)
ey_d = cuda.to_device(ey)
ez_d = cuda.to_device(ez)
bx_d = cuda.to_device(bx)
by_d = cuda.to_device(by)
bz_d = cuda.to_device(bz)

charge_d = cuda.to_device(charge)
xpos_d = cuda.to_device(xpos)
ypos_d = cuda.to_device(ypos)
zpos_d = cuda.to_device(zpos)
momentumx_d = cuda.to_device(momentumx)
momentumy_d = cuda.to_device(momentumy)
momentumz_d = cuda.to_device(momentumz)
mass_d = cuda.to_device(mass)
weighting_d = cuda.to_device(weighting)
chunk_offset_d = cuda.to_device(chunk_offset)

print("\nstart of field calculation time:", time.ctime())
starttime = time.time()

cudaFunc[griddim, blockdim](ex_d, ey_d, ez_d, bx_d, by_d, bz_d, charge_d, xpos_d, ypos_d, zpos_d, momentumx_d, momentumy_d, momentumz_d, mass_d, weighting_d, xdim, ydim, zdim, particleCount, chunk_offset_d)
cuda.synchronize()

exeTime = time.time()-starttime
print("time: {:.5}".format( exeTime ), "s")
print("avgTime per particle per cell:", exeTime / particleCount / (xdim*ydim*zdim), "s/(particle*cell)")


print("\nget field arrays from device", time.ctime())
ex_d.copy_to_host(ex)
ey_d.copy_to_host(ey)
ez_d.copy_to_host(ez)
bx_d.copy_to_host(bx)
by_d.copy_to_host(by)
bz_d.copy_to_host(bz)
print("field arrays copied from device to host", time.ctime())

# delete device variables, no longer needed after the parallel computation
del charge_d
del xpos_d
del ypos_d
del zpos_d
del momentumx_d
del momentumy_d
del momentumz_d
del mass_d
del weighting_d
del ex_d
del ey_d
del ez_d
del bx_d
del by_d
del bz_d

# write the calculate chunk of the field 
output_iteration.meshes['E']['x'].store_chunk(ex.reshape(xdim, ydim, zdim), chunk_offset, chunk_size)
output_iteration.meshes['E']['y'].store_chunk(ey.reshape(xdim, ydim, zdim), chunk_offset, chunk_size)
output_iteration.meshes['E']['z'].store_chunk(ez.reshape(xdim, ydim, zdim), chunk_offset, chunk_size)
output_iteration.meshes['B']['x'].store_chunk(bx.reshape(xdim, ydim, zdim), chunk_offset, chunk_size)
output_iteration.meshes['B']['y'].store_chunk(by.reshape(xdim, ydim, zdim), chunk_offset, chunk_size)
output_iteration.meshes['B']['z'].store_chunk(bz.reshape(xdim, ydim, zdim), chunk_offset, chunk_size)
files.write_series.flush()

print('done')

## When finished with all field chunks
you can write the other data to the file, except E, B fields that we wrote with this program

In [ ]:
del ex, ey, ez
del bx, by, bz

In [ ]:
# free memory for copy
del charge
del xpos
del ypos
del zpos
del momentumx
del momentumy
del momentumz
del mass
del weighting

In [ ]:
files.copy(exclude_mesh=['E', 'B'])

# Visualization of the fields if necessary

and other stuff, not realy documented with comments

## absolute field values

In [ ]:
# absolute values of the fields
ef = np.sqrt(ex**2+ey**2+ez**2)
bf = np.sqrt(bx**2+by**2+bz**2)

In [ ]:
plt.figure(figsize=(15,15))
depth = 380
plt.imshow(((ef.reshape(zdim, ydim, xdim)[depth,::4,:].T)))
#plt.xlim(290, 320)
#plt.ylim(160,200)
plt.colorbar()
plt.show()

In [ ]:
print(xpos, ypos, zpos)

## show particle positions

In [ ]:
plt.figure(figsize=(15,15))
a = np.histogram2d(xpos, ypos, weights=weightings, bins=[np.linspace(0, xdim, xdim+1), np.linspace(0, ydim, ydim+1)] )
plt.imshow(a[0])
plt.xlabel("y cells")
plt.ylabel("x cells")
plt.show()

In [ ]:
del ex_d
del ey_d
del ez_d
del bx_d
del by_d
del bz_d

In [ ]:
del writeSeries

## Further calculations for the generated data
the expected values are

\begin{equation}
\vec{\nabla} \cdot \vec{E} = \frac{\rho(\vec{r})}{\epsilon_0}
\end{equation}

\begin{equation}
\vec{\nabla} \times \vec{E} = 0
\end{equation}

\begin{equation}
\vec{\nabla} \cdot \vec{B} = 0
\end{equation}


In [ ]:
ex = ex.reshape(zdim, ydim, xdim)
ey = ey.reshape(zdim, ydim, xdim)
ez = ez.reshape(zdim, ydim, xdim)

bx = bx.reshape(zdim, ydim, xdim)
by = by.reshape(zdim, ydim, xdim)
bz = bz.reshape(zdim, ydim, xdim)

divE = ((ex[1:, 1:, 1:] - ex[1:, 1:, :-1]) / cell_width +
           (ey[1:, 1:, 1:] - ey[1:, :-1, 1:]) / cell_height +
           (ez[1:, 1:, 1:] - ez[:-1, 1:, 1:]) / cell_depth) * unit_efield / unit_length * scipy.constants.epsilon_0

divB = ((bx[1:, 1:, 1:] - bx[1:, 1:, :-1]) / cell_width +
       (by[1:, 1:, 1:] - by[1:, :-1, 1:]) / cell_height +
       (bz[1:, 1:, 1:] - bz[:-1, 1:, 1:]) / cell_depth) * unit_bfield * 299792458 / unit_length * scipy.constants.epsilon_0


In [ ]:
plt.figure(figsize=(20,20))
field = divE[:, :, 374]
plt.imshow(np.log(np.abs(field)))
#plt.xlim(200,400)
#plt.ylim(300,450)
plt.colorbar()

In [ ]:
plt.figure(figsize=(15,15))
plt.imshow(unit_efield*ez[:,:,128])
plt.colorbar()
plt.xlim(120,140)
plt.ylim(120,140)

In [ ]:
momentumy

In [ ]:
xx,yy,zz = 50,50,50    # width height depth - x y z
curlEx = np.zeros_like(ex)
curlEy = np.zeros_like(ex)
curlEz = np.zeros_like(ex)
curlBx = np.zeros_like(ex)
curlBy = np.zeros_like(ex)
curlBz = np.zeros_like(ex)

#for x in np.arange(100, 160):
#    for y in np.arange(100, 160):
#        for z in np.arange(100, 160):
#            curlEx[z,y,x] = (ez[z, y+1, x] - ez[z, y, x]) / cell_height - (ey[z+1, y, x] - ey[z, y, x]) / cell_depth
#            curlEy[z,y,x] = (ex[z+1, y, x] - ex[z, y, x]) / cell_depth  - (ez[z, y, x+1] - ez[z, y, x]) / cell_width
#            curlEz[z,y,x] = (ey[z, y, x+1] - ey[z, y, x]) / cell_width  - (ex[z, y+1, x] - ex[z, y, x]) / cell_height
#
#            curlBx[z,y,x] = (bz[z, y, x] - bz[z, y-1, x]) / cell_height - (by[z, y, x] - by[z-1, y, x]) / cell_depth
#            curlBy[z,y,x] = (bx[z, y, x] - bz[z-1, y, x]) / cell_depth  - (bz[z, y, x] - bz[z, y, x-1]) / cell_width
#            curlBz[z,y,x] = (by[z, y, x] - bz[z, y, x-1]) / cell_width  - (bx[z, y, x] - by[z, y-1, x]) / cell_height


curlEx = (ez[:-1, 1:, :-1] - ez[:-1, :-1, :-1]) / cell_height - (ey[1:, :-1, :-1] - ey[:-1, :-1, :-1]) / cell_depth
curlEy = (ex[1:, :-1, :-1] - ex[:-1, :-1, :-1]) / cell_depth  - (ez[:-1, :-1, 1:] - ez[:-1, :-1, :-1]) / cell_width
curlEz = (ey[:-1, :-1, 1:] - ey[:-1, :-1, :-1]) / cell_width  - (ex[:-1, 1:, :-1] - ex[:-1, :-1, :-1]) / cell_height
curlBx = (bz[1:, 1:, 1:] - bz[1:, :-1, 1:]) / cell_height - (by[1:, 1:, 1:] - by[:-1, 1:, 1:]) / cell_depth
curlBy = (bx[1:, 1:, 1:] - bz[:-1, 1:, 1:]) / cell_depth  - (bz[1:, 1:, 1:] - bz[1:, 1:, :-1]) / cell_width
curlBz = (by[1:, 1:, 1:] - bz[1:, 1:, :-1]) / cell_width  - (bx[1:, 1:, 1:] - by[1:, :-1, 1:]) / cell_height

In [ ]:
plt.figure(figsize=(10,10))
field = curlEz[374,:,:].T
plt.imshow(((field)))
plt.xlim(250,350)
plt.ylim(300,500)
plt.colorbar()

In [ ]:
ex.shape
e = ex.copy()

In [ ]:
plt.figure(figsize=(17,15))
field = divE[:,320,:]#np.sum(divE, axis=1)
plt.imshow(np.log(np.abs(field)))
plt.colorbar()

In [ ]:
dx = (ex[1:,1:,1:] - ex[1:,:-1,1:]) / cell_width

In [ ]:
plt.figure(figsize=(17,15))
field = divE[:,320,:]#np.sum(divE, axis=1)
plt.imshow(np.log(np.abs(field)))
plt.colorbar()

In [ ]:
plt.figure(figsize=(17,15))
plt.plot(dx[380,320,:])
x = np.linspace(0,800,800)
plt.plot(x, np.sign(x-374)*1e-10*(1/(x-374)**2))
plt.ylim(-7e-12, 7e-12)
#for i in np.arange(320, 400):
#    plt.plot(dx[i,320,:])